# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from feature_engine.outliers import Winsorizer 
from scipy.stats import kendalltau, pearsonr 
from sklearn.preprocessing import MinMaxScaler, StandardScaler,RobustScaler  
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder 
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, f1_score 
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve 
from sklearn.utils import shuffle 
from sklearn.compose import ColumnTransformer, make_column_selector as selector 
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
# from sklearn.preprocessing import LabelEncoder

# Loading data

In [3]:
# Import data
df = pd.read_csv('../Data_Clean/balikpapan_clean.csv')
df

,harga,lokasi,luas_bangunan,luas_tanah,sertifikat,furnish,kamar_tidur,kamar_mandi,listrik,lantai
0,1650000,Balikpapan Selatan,180.0,141,SHM,Unfurnished,4,3,2200,2
1,2100000,Balikpapan Selatan,50.0,776,SHM,Unfurnished,3,1,900,1
2,1950000,Balikpapan Utara,139.7,206,SHM,Unfurnished,5,2,2200,1
3,1180000,Balikpapan Utara,47.0,78,SHM,Semi-furnished,2,2,1300,1
4,2500000,Balikpapan Baru,160.0,169,HGB,Unfurnished,4,3,4400,1
...,...,...,...,...,...,...,...,...,...,...
1569,500000,Balikpapan Selatan,42.0,84,HGB,Furnished,2,1,1300,1
1570,807000,Balikpapan Selatan,50.0,90,HGB,Unfurnished,3,2,2200,1
1571,480000,Balikpapan Selatan,48.0,72,SHM,Unfurnished,2,1,1300,1
1572,8500000,Balikpapan Baru,380.0,520,SHM,Unfurnished,4,2,6600,1


In [4]:
# Info data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   harga          1574 non-null   int64  
 1   lokasi         1574 non-null   object 
 2   luas_bangunan  1574 non-null   float64
 3   luas_tanah     1574 non-null   int64  
 4   sertifikat     1574 non-null   object 
 5   furnish        1574 non-null   object 
 6   kamar_tidur    1574 non-null   int64  
 7   kamar_mandi    1574 non-null   int64  
 8   listrik        1574 non-null   int64  
 9   lantai         1574 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 123.1+ KB


In [6]:
pd.set_option('display.max_colwidth', None)

cat_cols = list(df.select_dtypes(include='object').columns)

listItem = []
for col in cat_cols:
  listItem.append([col, df[col].nunique()])

pd.DataFrame(columns=['nama kolom', 'jumlah nilai unique'], data=listItem)

,nama kolom,jumlah nilai unique
0,lokasi,25
1,sertifikat,2
2,furnish,3


In [7]:
pd.set_option('display.max_colwidth', None)

cat_cols = (df.select_dtypes(include=['int64', 'float64']).columns)

listItem = []
for col in cat_cols:
    listItem.append([col, df[col].nunique()])

result_df = pd.DataFrame(listItem, columns=['nama kolom', 'jumlah nilai unique'])
result_df


,nama kolom,jumlah nilai unique
0,harga,290
1,luas_bangunan,230
2,luas_tanah,284
3,kamar_tidur,10
4,kamar_mandi,8
5,listrik,22
6,lantai,2


In [9]:
# print unique lokasi
df['lokasi'].unique()

array(['Balikpapan Selatan', 'Balikpapan Utara', 'Balikpapan Baru',
       'Damai', 'Karang Rejo', 'Sepinggan', 'Gunung Bahagia',
       'Balikpapan Tengah', 'Balikpapan Timur', 'Batu Ampar',
       'Balikpapan Barat', 'Muara Rapak', 'Lamaru', 'Telaga Sari',
       'Manggar', 'Karang Jati', 'Kariangau', 'Gn. Samarinda',
       'Sumber Rejo', 'Klandasan Ulu', 'Klandasan Ilir', 'Prapatan',
       'Batakan', 'Gn. Sari Ilir', 'Balikpapan Kota'], dtype=object)

In [10]:
# check min max value of all numeric column
df.describe()

,harga,luas_bangunan,luas_tanah,kamar_tidur,kamar_mandi,listrik,lantai
count,1.574000e+03,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000,1574.000000
mean,1.990419e+06,165.968742,219.142948,3.389454,2.346887,2841.677255,1.213469
std,1.792939e+06,121.133668,199.580171,1.279270,1.158895,2552.149783,0.409886
min,1.250000e+05,8.000000,36.000000,0.000000,0.000000,900.000000,1.000000
25%,8.550000e+05,88.000000,120.000000,2.000000,2.000000,1300.000000,1.000000
50%,1.500000e+06,130.000000,162.000000,3.000000,2.000000,2200.000000,1.000000
75%,2.500000e+06,212.250000,243.000000,4.000000,3.000000,3500.000000,1.000000
max,1.500000e+07,1270.000000,4300.000000,9.000000,7.000000,53000.000000,2.000000


# Split

In [11]:
X = df.drop(['harga'], axis = 1)
y = df['harga']
X

,lokasi,luas_bangunan,luas_tanah,sertifikat,furnish,kamar_tidur,kamar_mandi,listrik,lantai
0,Balikpapan Selatan,180.0,141,SHM,Unfurnished,4,3,2200,2
1,Balikpapan Selatan,50.0,776,SHM,Unfurnished,3,1,900,1
2,Balikpapan Utara,139.7,206,SHM,Unfurnished,5,2,2200,1
3,Balikpapan Utara,47.0,78,SHM,Semi-furnished,2,2,1300,1
4,Balikpapan Baru,160.0,169,HGB,Unfurnished,4,3,4400,1
...,...,...,...,...,...,...,...,...,...
1569,Balikpapan Selatan,42.0,84,HGB,Furnished,2,1,1300,1
1570,Balikpapan Selatan,50.0,90,HGB,Unfurnished,3,2,2200,1
1571,Balikpapan Selatan,48.0,72,SHM,Unfurnished,2,1,1300,1
1572,Balikpapan Baru,380.0,520,SHM,Unfurnished,4,2,6600,1
